In [ ]:
# default_exp data.features

# Featurizing Time Series

> Functions used to transform time series into a dataframe that can be used to create tabular dataloaders.

In [ ]:
#export
from tsai.imports import *
from tsai.utils import *
from tsai.data.external import *
from tsai.data.core import *
from tsai.data.tabular import *

In this case we are using tsfresh that is one of the most widely known libraries used to create features from time series. You can get more details about this library here: https://tsfresh.readthedocs.io/en/latest/

In [ ]:
#export
import tsfresh
from tsfresh import extract_features
from tsfresh.feature_extraction.settings import ComprehensiveFCParameters, MinimalFCParameters, EfficientFCParameters

In [ ]:
#export
@delegates(extract_features.__init__)
def get_ts_features(X:Union[np.ndarray, torch.Tensor], y:Union[None, np.ndarray, torch.Tensor]=None, features:Union[str, dict]='min', 
                    n_jobs:Optional[int]=None, **kwargs):
    """
    Args:
        X: np.array or torch.Tesnor of shape [samples, dimensions, timesteps].
        y: Not required for unlabeled data. Otherwise, you need to pass it.
        features: 'min', 'efficient', 'all', or a dictionary. Be aware that 'efficient' and 'all' may required substantial memory and time.
    """
    df = to_tsfresh_df(X)
    n_jobs = ifnone(n_jobs, defaults.cpus)
    if 'default_fc_parameters' in kwargs.keys(): default_fc_parameters = default_fc_parameters 
    elif features == 'min': default_fc_parameters = MinimalFCParameters()
    elif features == 'efficient': default_fc_parameters = EfficientFCParameters()
    elif features == 'all': default_fc_parameters = ComprehensiveFCParameters()
    else: default_fc_parameters = None
    df = tsfresh.extract_features(df, column_id="id", n_jobs=n_jobs, default_fc_parameters=default_fc_parameters, **kwargs)
    if y is not None:
        if y.ndim == 1: y = y.reshape(-1,1)
        for i in range(y.shape[-1]):
            df['target' if y.shape[-1] == 1 else f'target_{i}'] = y[:, i]
    return df

In [ ]:
dsid = 'NATOPS'
X, y, splits = get_UCR_data(dsid, return_split=False)
X.shape

(360, 24, 51)

There are 3 levels of fatures you can extract: 'min', 'efficient' and 'all'. I'd encourage you to start with min as feature creation may take a long time. 

In addition to this, you can pass a dictionary to build the desired features (see tsfresh documentation in the link above).

In [ ]:
ts_features_df = get_ts_features(X, y)
ts_features_df.shape

Feature Extraction: 100%|██████████| 40/40 [00:05<00:00,  7.58it/s]


(360, 193)

The 'min' set creates a dataframe with 8 features per channel + 1 per target (total 193) for each time series sample (360).

In [ ]:
cont_names = ts_features_df.columns[:-1]
y_names = 'target'
dls = get_tabular_dls(ts_features_df, splits=splits, cont_names=cont_names, y_names=y_names)
dls.show_batch()

,0__sum_values,0__median,0__mean,0__length,0__standard_deviation,0__variance,0__maximum,0__minimum,1__sum_values,1__median,1__mean,1__length,1__standard_deviation,1__variance,1__maximum,1__minimum,2__sum_values,2__median,2__mean,2__length,2__standard_deviation,2__variance,2__maximum,2__minimum,3__sum_values,3__median,3__mean,3__length,3__standard_deviation,3__variance,3__maximum,3__minimum,4__sum_values,4__median,4__mean,4__length,4__standard_deviation,4__variance,4__maximum,4__minimum,5__sum_values,5__median,5__mean,5__length,5__standard_deviation,5__variance,5__maximum,5__minimum,6__sum_values,6__median,6__mean,6__length,6__standard_deviation,6__variance,6__maximum,6__minimum,7__sum_values,7__median,7__mean,7__length,7__standard_deviation,7__variance,7__maximum,7__minimum,8__sum_values,8__median,8__mean,8__length,8__standard_deviation,8__variance,8__maximum,8__minimum,9__sum_values,9__median,9__mean,9__length,9__standard_deviation,9__variance,9__maximum,9__minimum,10__sum_values,10__median,10__mean,10__length,10__standard_deviation,10__variance,10__maximum,10__minimum,11__sum_values,11__median,11__mean,11__length,11__standard_deviation,11__variance,11__maximum,11__minimum,12__sum_values,12__median,12__mean,12__length,12__standard_deviation,12__variance,12__maximum,12__minimum,13__sum_values,13__median,13__mean,13__length,13__standard_deviation,13__variance,13__maximum,13__minimum,14__sum_values,14__median,14__mean,14__length,14__standard_deviation,14__variance,14__maximum,14__minimum,15__sum_values,15__median,15__mean,15__length,15__standard_deviation,15__variance,15__maximum,15__minimum,16__sum_values,16__median,16__mean,16__length,16__standard_deviation,16__variance,16__maximum,16__minimum,17__sum_values,17__median,17__mean,17__length,17__standard_deviation,17__variance,17__maximum,17__minimum,18__sum_values,18__median,18__mean,18__length,18__standard_deviation,18__variance,18__maximum,18__minimum,19__sum_values,19__median,19__mean,19__length,19__standard_deviation,19__variance,19__maximum,19__minimum,20__sum_values,20__median,20__mean,20__length,20__standard_deviation,20__variance,20__maximum,20__minimum,21__sum_values,21__median,21__mean,21__length,21__standard_deviation,21__variance,21__maximum,21__minimum,22__sum_values,22__median,22__mean,22__length,22__standard_deviation,22__variance,22__maximum,22__minimum,23__sum_values,23__median,23__mean,23__length,23__standard_deviation,23__variance,23__maximum,23__minimum,target
0,-20.952641,-0.405189,-0.410836,51.0,0.018057,0.000326,-0.384822,-0.449682,-109.506318,-2.168418,-2.147183,51.0,0.049802,0.002480,-2.032201,-2.242009,-47.214524,-0.922208,-0.925775,51.0,0.033525,0.001124,-0.876040,-1.007949,59.958053,0.621453,1.175648,51.0,0.824611,0.679983,2.361367,0.384041,-72.868484,-2.175878,-1.428794,51.0,1.197618,1.434288,0.985820,-2.374636,-2.812251,-0.394236,-0.055142,51.0,0.537110,0.288487,0.876690,-0.547092,-37.752014,-0.739922,-0.740236,51.0,0.007544,0.000057,-0.729232,-0.755185,-37.262589,-0.736720,-0.730639,51.0,0.019667,0.000387,-0.695408,-0.750920,-21.131655,-0.411820,-0.414346,51.0,0.015462,0.000239,-0.390847,-0.450483,34.157357,0.516974,0.669752,51.0,0.216662,0.046943,1.041962,0.470650,-32.515407,-0.858229,-0.637557,51.0,0.354173,0.125438,0.120601,-1.028824,21.322239,0.282372,0.418083,51.0,0.196967,0.038796,0.734371,0.223754,-28.909897,-0.563849,-0.566861,51.0,0.029449,0.000867,-0.528048,-0.628923,-82.734826,-1.667627,-1.622251,51.0,0.070965,0.005036,-1.472939,-1.707529,-40.479702,-0.808652,-0.793720,51.0,0.057730,0.003333,-0.697692,-0.915657,50.249493,0.632003,0.985284,51.0,0.533602,0.284731,1.765066,0.479747,-58.823982,-1.648608,-1.153411,51.0,0.861036,0.741382,0.559503,-1.870982,7.035380,-0.111731,0.137949,51.0,0.424846,0.180494,0.816254,-0.293276,-31.142656,-0.605069,-0.610640,51.0,0.023582,0.000556,-0.559798,-0.657762,-96.307694,-1.894914,-1.888386,51.0,0.056496,0.003192,-1.756580,-2.020149,-43.275879,-0.845870,-0.848547,51.0,0.064078,0.004106,-0.749379,-0.986353,

In [ ]:
x_cat, x_cont, yb = first(dls.train)
x_cont

tensor([[-0.6617, -0.8070, -0.6617,  ..., -1.0240, -1.4602,  0.3117],
        [ 2.7011,  1.8840,  2.7011,  ..., -0.4987, -0.3468,  0.6616],
        [-0.0764, -0.1685, -0.0764,  ..., -0.1442, -0.1120, -0.8724],
        ...,
        [-0.9232, -0.1029, -0.9232,  ..., -0.1067, -0.1817, -2.0275],
        [ 1.9545,  0.7020,  1.9545,  ..., -0.1624, -0.6785, -1.0776],
        [ 1.1814,  0.4320,  1.1814,  ..., -0.5130, -0.6362,  0.1105]])

In [ ]:
from tsai.models.utils import *
from tsai.models.TabModel import *
model = build_tabular_model(TabModel, dls=dls)
learn = Learner(dls, model, metrics=[accuracy, RocAuc()])
learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,time
0,1.807627,1.777147,0.172222,0.632667,00:00
1,1.779090,1.665929,0.361111,0.911815,00:00
2,1.725221,1.554959,0.544444,0.940852,00:00
3,1.669523,1.479427,0.605556,0.943926,00:00
4,1.622043,1.457247,0.627778,0.944333,00:00


In [ ]:
b = first(dls.train)
model(*b[:-1]).shape

torch.Size([64, 6])

In [ ]:
#hide
out = create_scripts(); beep(out)

<IPython.core.display.Javascript object>

Converted 000_utils.ipynb.
Converted 000b_data.validation.ipynb.
Converted 000c_data.preparation.ipynb.
Converted 001_data.external.ipynb.
Converted 002_data.core.ipynb.
Converted 003_data.preprocessing.ipynb.
Converted 003b_data.transforms.ipynb.
Converted 003c_data.mixed_augmentation.ipynb.
Converted 003d_data.image.ipynb.
Converted 003e_data.features.ipynb.
Converted 005_data.tabular.ipynb.
Converted 006_data.mixed.ipynb.
Converted 007_metrics.ipynb.
Converted 008_learner.ipynb.
Converted 009_optimizer.ipynb.
Converted 010_callback.core.ipynb.
Converted 011_callback.semi_supervised.ipynb.
Converted 100_models.utils.ipynb.
Converted 100b_models.layers.ipynb.
Converted 100c_models.explainability.ipynb.
Converted 101_models.ResNet.ipynb.
Converted 101b_models.ResNetPlus.ipynb.
Converted 102_models.InceptionTime.ipynb.
Converted 102b_models.InceptionTimePlus.ipynb.
Converted 103_models.MLP.ipynb.
Converted 103b_models.FCN.ipynb.
Converted 103c_models.FCNPlus.ipynb.
Converted 104_models.